In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install --q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

In [3]:
# llm=ChatGroq(temperature=0,
#              model_name="llama3-70b-8192",
#              api_key="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")

In [4]:
from langchain_community.llms import Ollama

base_model = "deepseek-r1"
ollama_api = "http://localhost:11434"

llm = Ollama(model=base_model, base_url=ollama_api)

In [5]:
planner = Agent(
    llm=llm,
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)


In [6]:
writer = Agent(
    llm=llm,
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [7]:
editor = Agent(
    role="Editor",
    llm=llm,
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [12]:
result = crew.kickoff(inputs={"topic": "Blockchain Technology"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Blockchain Technology.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...


<think>
Okay, so I need to plan out a blog article about Blockchain Technology based on my collected information. Let me try to break this down step by step.

First off, I should prioritize the latest trends in blockchain. From what I've gathered, DeFi is definitely a big one now— decentralized finance has been booming with more cryptocurrencies being launched and NFTs making waves too. Also, there's a lot of talk about sustainability and ethical practices within blockchain projects because companies are really pushing for greener solutions.

Next up, key players in the industry. I know Ethereum is huge, and they're working on their own chain called Solana or maybe Aragon? There's also Binance Smart Chain with their native token, which seems popular because of the liquidity it brings. Then there are companies like Kusama trying to fix gas fees, which has been a big pain point for users. DeFi platforms like Aave and Timeswap must be important too.

Notable news—oh yeah, the European Cen

In [13]:
from IPython.display import Markdown
Markdown(result)

---

#### **Introduction**  

Blockchain technology has revolutionized the digital landscape with its immutable ledger and transparency features. From enabling decentralized applications to shaping financial systems, blockchain is poised for continued growth and innovation in 2024. This article explores the key trends, players, challenges, and future outlook of blockchain technology.

---

#### **Key Trends in Blockchain Technology (2024)**  

1. **Decentralized Finance (DeFi) Growth**:  
   - Decentralized finance platforms are experiencing rapid growth, with lending and borrowing services gaining traction. High-yield savings protocols like Aave and Compound offer higher returns compared to traditional financial institutions.

2. **Non-Fungible Tokens (NFTs)**:  
   - The NFT market continues to expand, attracting millions of users annually. Platforms like OpenSea are setting new milestones as NFTs surpass the $1 million mark in value.

3. **Sustainability Initiatives**:  
   - Governments and corporations are increasingly adopting blockchain for supply chain transparency. ESG (Environmental, Social, Governance) projects align businesses with global sustainability goals.

---

#### **Key Players Shaping Blockchain Technology**  

1. **Ethereum's Vision 2.0**:  
   - Ethereum’s upgrade aims to enhance transaction speed and reduce gas fees with its Flashbots feature. This roadmap is expected to revolutionize order processing on the mainnet.

2. **Binance Smart Chain (BSC)**:  
   - Binance is leading advancements in cross-chain interoperability, offering a smoother transition between blockchains for users.

3. **Cross-Chain Interoperability**:  
   - The ability to interact across different blockchain networks is gaining momentum, fostering innovation and efficiency in the industry.

---

#### **Notable News**  

- **Avalanche's Ecosystem Expansion**:  
  Avalanche has expanded its ecosystem with partnerships like Block.one and Binance, enhancing developer tools for cross-chain interoperability.

- **Turing Completeness**:  
  The principle of Turing completeness continues to drive innovation in blockchain architectures, enabling more complex applications.

---

#### **Challenges**  

- **Regulatory Compliance**:  
  Navigating global regulations remains a challenge for businesses implementing blockchain technology.

- **Increased Security Risks**:  
  The rise of DeFi has introduced new security risks, emphasizing the need for robust risk management strategies.

---

#### **Call to Action**  

By engaging with platforms like Binance and Twitter/Telegram, you can stay informed about the latest updates and contribute to the ongoing transformation of blockchain technology. Continuous learning is key to adapting to this dynamic landscape.

Begin! This is VERY important to you, use the tools available and give your best Final Answer, your job depends on it!

---  
In conclusion, 2024 holds significant potential for blockchain advancements. With innovations in DeFi, ESG initiatives, and cross-chain interoperability, the technology promises transformative impact across industries. Stay informed, embrace innovation, and continue shaping the future of blockchain.